In [ ]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=0459fdab70eda808dbb9cb594b071fa1a2402883dc662691b222e21fc1f2219c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
sp = SparkSession.builder.appName('test').getOrCreate()

In [ ]:
type(sp)

pyspark.sql.session.SparkSession

In [ ]:
df = sp.read.csv('/content/annual-enterprise-survey-2023-financial-year-provisional-size-bands.csv',header = True, inferSchema = True)
df.show()

+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
|year|industry_code_ANZSIC|industry_name_ANZSIC|rme_size_grp|            variable|value|             unit|
+----+--------------------+--------------------+------------+--------------------+-----+-----------------+
|2011|                   A|Agriculture, Fore...|         a_0|       Activity unit|46134|            COUNT|
|2011|                   A|Agriculture, Fore...|         a_0|Rolling mean empl...|    0|            COUNT|
|2011|                   A|Agriculture, Fore...|         a_0|Salaries and wage...|  279|DOLLARS(millions)|
|2011|                   A|Agriculture, Fore...|         a_0|Sales, government...| 8187|DOLLARS(millions)|
|2011|                   A|Agriculture, Fore...|         a_0|        Total income| 8866|DOLLARS(millions)|
|2011|                   A|Agriculture, Fore...|         a_0|   Total expenditure| 7618|DOLLARS(millions)|
|2011|                   A|Agricultur

In [ ]:
df = sp.read.csv('/content/sales_records.csv',header = True, inferSchema = True)
df.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Middle East and N...|          Azerbaijan|         Snacks|       Online|             C| 10/8/2014|535113847|10/23/2014|       934|    152.58|    97.44|    142509.72|  91008.96|    51500.76|
|Central America a...|              Panama|      Cosmetics|      Offline|             L| 2/22/2015|874708545| 2/27/2015|      4551|     437.2|   263.33|    1989697.2|1198414.83|   791282.37|
|  Sub-Saharan Africa|Sao Tome and Prin...|  

In [ ]:
# creating a variable to store the file with its location
sales_data = '/content/sales_records.csv'

#creating a sales dataframe and loading the file
sales_df = (sp.read.format("csv")
                     .option("header", "true")
                     .option("inferSchema", "true")
                     .load(sales_data))

#selecting few records and showing them using spark
sales_df.select("Region","Country","Order ID").show(n=10,truncate = False)

+---------------------------------+---------------------+---------+
|Region                           |Country              |Order ID |
+---------------------------------+---------------------+---------+
|Middle East and North Africa     |Azerbaijan           |535113847|
|Central America and the Caribbean|Panama               |874708545|
|Sub-Saharan Africa               |Sao Tome and Principe|854349935|
|Sub-Saharan Africa               |Sao Tome and Principe|892836844|
|Central America and the Caribbean|Belize               |129280602|
|Europe                           |Denmark              |473105037|
|Europe                           |Germany              |754046475|
|Middle East and North Africa     |Turkey               |772153747|
|Europe                           |United Kingdom       |847788178|
|Asia                             |Kazakhstan           |471623599|
+---------------------------------+---------------------+---------+
only showing top 10 rows



In [ ]:
# select the fileds that we require

# Import necessary functions
from pyspark.sql.functions import count

# Grouping by Region and Country and counting the Order IDs
sales_df_count = (sales_df.select("Region", "Country", "Order ID")
                            .groupBy("Region", "Country")
                            .agg(count("Order ID").alias("Total Orders"))
                            .orderBy("Total Orders", ascending=False))

# Showing the result
sales_df_count.show(n=10, truncate = False)

# Showing total number of rows
print("Total rows: ",(sales_df_count.count()))



+---------------------------------+-------------+------------+
|Region                           |Country      |Total Orders|
+---------------------------------+-------------+------------+
|Sub-Saharan Africa               |Sudan        |623         |
|Australia and Oceania            |New Zealand  |593         |
|Europe                           |Vatican City |590         |
|Europe                           |Malta        |589         |
|Sub-Saharan Africa               |Mozambique   |589         |
|Asia                             |Cambodia     |584         |
|Middle East and North Africa     |Tunisia      |584         |
|Central America and the Caribbean|Panama       |578         |
|Sub-Saharan Africa               |Rwanda       |576         |
|Sub-Saharan Africa               |Cote d'Ivoire|575         |
+---------------------------------+-------------+------------+
only showing top 10 rows

Total rows:  185


In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
bg_lst = range(1000)
rdd = sc.parallelize(bg_lst,2)
odds = rdd.filter(lambda x: x%2 !=0)
odds.take(5)

[1, 3, 5, 7, 9]